In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')

In [4]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [5]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5 OFFSET 10
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.22489,12.5,7.87,0.0,0.524,6.377,94.3,6.3467,5.0,311.0,15.2,392.52,20.45,15.0
1,0.11747,12.5,7.87,0.0,0.524,6.009,82.9,6.2267,5.0,311.0,15.2,396.90,13.27,18.9
2,0.09378,12.5,7.87,0.0,0.524,5.889,39.0,5.4509,5.0,311.0,15.2,390.50,15.71,21.7
3,0.62976,0.0,8.14,0.0,0.538,5.949,61.8,4.7075,4.0,307.0,21.0,396.90,8.26,20.4
4,0.63796,0.0,8.14,0.0,0.538,6.096,84.5,4.4619,4.0,307.0,21.0,380.02,10.26,18.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [6]:
pd.read_sql(
    """
    SELECT 'CRIM' AS 'variable', SUM(CASE WHEN CRIM IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'ZN', SUM(CASE WHEN ZN IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'INDUS', SUM(CASE WHEN INDUS IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'CHAS', SUM(CASE WHEN CHAS IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston
     UNION ALL
    SELECT 'NOX', SUM(CASE WHEN NOX IS NULL THEN 1 ELSE 0 END) AS nulls_count
      FROM boston;
    """,
    con,
)

,variable,nulls_count
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


### Вывод:
В данных нет пропущенных значений

2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [7]:
pd.read_sql(
    """
    SELECT 'CRIM' AS 'variable', COUNT(CRIM) AS unique_count
      FROM (SELECT DISTINCT CRIM FROM boston)
     UNION ALL
    SELECT 'ZN', COUNT(ZN) as unique_count
      FROM (SELECT DISTINCT ZN FROM boston)
     UNION ALL
    SELECT 'INDUS', COUNT(INDUS) AS unique_count
      FROM (SELECT DISTINCT INDUS FROM boston)
     UNION ALL
    SELECT 'CHAS', COUNT(CHAS) AS unique_count
      FROM (SELECT DISTINCT CHAS FROM boston)
     UNION ALL
    SELECT 'NOX', COUNT(NOX) AS unique_count
      FROM (SELECT DISTINCT NOX FROM boston);
    """,
    con,
)

,variable,unique_count
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


### Вывод:
В данных есть бинарная переменная CHAS. Самые разнообразные данные по криминальной составляющей CRIM.

3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [8]:
pd.read_sql(
    """
    SELECT parameter AS median_eq_min FROM (
    SELECT 'CRIM' AS parameter,
           AVG(CRIM) AS median,
           (SELECT MIN(CRIM) FROM boston) AS min_val
      FROM (SELECT CRIM
              FROM boston
          ORDER BY CRIM
             LIMIT 2 - (SELECT COUNT(CRIM) FROM boston) % 2
            OFFSET (SELECT COUNT(CRIM) FROM boston) / 2)
    UNION ALL
    SELECT 'ZN' AS parameter,
           AVG(ZN) AS median,
           (SELECT MIN(ZN) FROM boston) AS min_val
      FROM (SELECT ZN
              FROM boston
          ORDER BY ZN
             LIMIT 2 - (SELECT COUNT(ZN) FROM boston) % 2
            OFFSET (SELECT COUNT(ZN) FROM boston) / 2)
    UNION ALL
    SELECT 'INDUS' AS parameter,
           AVG(INDUS) AS median,
           (SELECT MIN(INDUS) FROM boston) AS min_val
      FROM (SELECT INDUS
              FROM boston
          ORDER BY INDUS
             LIMIT 2 - (SELECT COUNT(INDUS) FROM boston) % 2
            OFFSET (SELECT COUNT(INDUS) FROM boston) / 2)
    UNION ALL
    SELECT 'CHAS' AS parameter,
           AVG(CHAS) AS median,
           (SELECT MIN(CHAS) FROM boston) AS min_val
      FROM (SELECT CHAS
              FROM boston
          ORDER BY CHAS
             LIMIT 2 - (SELECT COUNT(CHAS) FROM boston) % 2
            OFFSET (SELECT COUNT(CHAS) FROM boston) / 2)
    UNION ALL
    SELECT 'NOX' AS parameter,
           AVG(NOX) AS median,
           (SELECT MIN(NOX) FROM boston) AS min_val
      FROM (SELECT NOX
              FROM boston
          ORDER BY NOX
             LIMIT 2 - (SELECT COUNT(NOX) FROM boston) % 2
            OFFSET (SELECT COUNT(NOX) FROM boston) / 2)
        )
    WHERE median = min_val;
    """,
    con,
)

,median_eq_min
0,ZN
1,CHAS


### Вывод:
Больше половины значений переменных ZN и CHAS равны минимальному значению. CHAS - это бинарная переменная и логично, что бОльшая часть домов не находится рядом с рекой. ZN - видимо бОльшая часть домов не находятся на участке зонированом под застройку.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [9]:
pd.read_sql(
    """
    SELECT 'lowest 25' AS cheap_range, *, rm_exp - rm_cheap diff
      FROM (SELECT AVG(RM) AS rm_exp,
                   (SELECT AVG(RM)
                      FROM (SELECT MEDV, RM
                              FROM boston
                          ORDER BY MEDV ASC
                             LIMIT 25)
                   ) AS rm_cheap
              FROM boston
          GROUP BY MEDV
          ORDER BY MEDV DESC
             LIMIT 1)
    UNION ALL
    SELECT 'lowest 50' AS cheap_range, *, rm_exp - rm_cheap diff
      FROM (SELECT AVG(RM) AS rm_exp,
                   (SELECT AVG(RM)
                      FROM (SELECT MEDV, RM
                              FROM boston
                          ORDER BY MEDV ASC
                             LIMIT 50)
                   ) AS rm_cheap
              FROM boston
          GROUP BY MEDV
          ORDER BY MEDV DESC
             LIMIT 1)
    UNION ALL
    SELECT 'lowest 100' AS cheap_range, *, rm_exp - rm_cheap diff
      FROM (SELECT AVG(RM) AS rm_exp,
                   (SELECT AVG(RM)
                      FROM (SELECT MEDV, RM
                              FROM boston
                          ORDER BY MEDV ASC
                             LIMIT 100)
                   ) AS rm_cheap
              FROM boston
          GROUP BY MEDV
          ORDER BY MEDV DESC
             LIMIT 1)
    UNION ALL
    SELECT 'lowest 200' AS cheap_range, *, rm_exp - rm_cheap diff
      FROM (SELECT AVG(RM) AS rm_exp,
                   (SELECT AVG(RM)
                      FROM (SELECT MEDV, RM
                              FROM boston
                          ORDER BY MEDV ASC
                             LIMIT 200)
                   ) AS rm_cheap
              FROM boston
          GROUP BY MEDV
          ORDER BY MEDV DESC
             LIMIT 1)
    UNION ALL
    SELECT 'lowest 300' AS cheap_range, *, rm_exp - rm_cheap diff
      FROM (SELECT AVG(RM) AS rm_exp,
                   (SELECT AVG(RM)
                      FROM (SELECT MEDV, RM
                              FROM boston
                          ORDER BY MEDV ASC
                             LIMIT 300)
                   ) AS rm_cheap
              FROM boston
          GROUP BY MEDV
          ORDER BY MEDV DESC
             LIMIT 1);
    """,
    con,
)

,cheap_range,rm_exp,rm_cheap,diff
0,lowest 25,7.484,5.747840,1.736160
1,lowest 50,7.484,5.753240,1.730760
2,lowest 100,7.484,5.887120,1.596880
3,lowest 200,7.484,5.911705,1.572295
4,lowest 300,7.484,5.972227,1.511773


### Вывод:
Стоимость дома незначительно зависит от количества комнат в нем, но связь несильная, т.к. во всех домах от 5 до 8 комнат.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [10]:
pd.read_sql(
    """
    SELECT *
      FROM (SELECT LSTAT, MEDV, rank() OVER (ORDER BY LSTAT DESC) AS rank_by_LSTAT
              FROM boston)
     WHERE MEDV = (SELECT MAX(MEDV) FROM boston);
    """,
    con,
)

,LSTAT,MEDV,rank_by_LSTAT
0,9.53,50.0,304
1,8.88,50.0,326
2,7.44,50.0,365
3,5.12,50.0,440
4,4.63,50.0,457
5,4.45,50.0,465
6,3.73,50.0,480
7,3.70,50.0,481
8,3.32,50.0,489
9,3.26,50.0,490


### Вывод:
В дорогих домах живут люди из благополучных семей.

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [12]:
pd.read_sql(
    """
      SELECT CASE
                WHEN CHAS = 1 THEN 'Yes'
                ELSE 'No'
             END AS near_river,
             AVG(MEDV) AS avg_price_kUSD
        FROM boston
    GROUP BY CHAS;
    """,
    con,
)

,near_river,avg_price_kUSD
0,No,22.093843
1,Yes,28.440000


### Вывод:
Около реки дома в среднем дороже

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [11]:
pd.read_sql(
    """
    SELECT parameter
      FROM (SELECT 'CRIM' AS parameter,
                   (SELECT AVG(CRIM)
                      FROM boston
                     WHERE CHAS = 1
                  GROUP BY CHAS) AS near,
                   (SELECT AVG(CRIM)
                      FROM boston
                     WHERE CHAS = 0
                  GROUP BY CHAS) AS far
            UNION ALL
            SELECT 'ZN' AS parameter,
                   (SELECT AVG(ZN)
                      FROM boston
                     WHERE CHAS = 1
                  GROUP BY CHAS) AS near,
                   (SELECT AVG(ZN)
                      FROM boston
                     WHERE CHAS = 0
                  GROUP BY CHAS) AS far
            UNION ALL
            SELECT 'INDUS' AS parameter,
                   (SELECT AVG(INDUS)
                      FROM boston
                     WHERE CHAS = 1
                  GROUP BY CHAS) AS near,
                   (SELECT AVG(INDUS)
                      FROM boston
                     WHERE CHAS = 0
                  GROUP BY CHAS) AS far
            UNION ALL
            SELECT 'NOX' AS parameter,
                   (SELECT AVG(NOX)
                      FROM boston
                     WHERE CHAS = 1
                  GROUP BY CHAS) AS near,
                   (SELECT AVG(NOX)
                      FROM boston
                     WHERE CHAS = 0
                  GROUP BY CHAS) AS far)
    WHERE near > far;
    """,
    con,
)

,parameter
0,INDUS
1,NOX


### Вывод:
Как видим рядом с рекой больше промышленных предприятий и следовательно большее содержание диоксида азота в воздухе.

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [13]:
pd.read_sql(
    """
    SELECT n.perc_name, indus_val, nox_val
      FROM (SELECT perc_name * 10 AS perc_name, indus_val
              FROM (SELECT perc_name,
                           LAST_VALUE(INDUS) OVER(PARTITION BY perc_name) AS indus_val
                      FROM (SELECT INDUS,
                                   NTILE(10) OVER(ORDER BY INDUS) AS perc_name
                              FROM boston))
          GROUP BY indus_val) AS n
      JOIN (SELECT perc_name * 10 AS perc_name, nox_val
              FROM (SELECT perc_name,
                           LAST_VALUE(NOX) OVER(PARTITION BY perc_name) AS nox_val
                      FROM (SELECT NOX,
                                   NTILE(10) OVER(ORDER BY NOX) AS perc_name
                              FROM boston))
          GROUP BY nox_val) AS i
        ON n.perc_name = i.perc_name;
    """,
    con,
)

,perc_name,indus_val,nox_val
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,13.89,0.580
6,70,18.10,0.609
7,90,19.58,0.713
8,100,27.74,0.871


### Вывод:
Как видим есть прямая связь доли промышленной застройки с содержанием диоксида азота в воздухе.